In [1]:
import sqlite3 as sq
import pandas as pd
import numpy as np
from sklearn import preprocessing as pp
from sklearn.cross_validation import cross_val_score as cv
from sklearn.grid_search import GridSearchCV as gscv
from sklearn.feature_selection import RFECV as rfe
from sklearn import linear_model as lm
from statsmodels.formula.api import logit
import matplotlib.pyplot as plt
from patsy import dmatrix, dmatrices
import matplotlib.pylab as plt
from pylab import rcParams

In [5]:
#LOAD USER-DEFINED FUNCTIONS FOR DATA MANIPULATION-----------------------------

#convert low-freq categorical feature values to 'Other'
def cleanup_data(df, cutoffPercent = .01):
    for col in df:
        sizes = df[col].value_counts(normalize = True)
        values_to_delete = sizes[sizes<cutoffPercent].index
        df[col].ix[df[col].isin(values_to_delete)] = "Other"
    return df

#binazrize catergoical feature values into individual variables
def get_binary_values(data_frame):
    """encodes categorical features in Pandas."""
    all_columns = pd.DataFrame(index = data_frame.index)
    for col in data_frame.columns:
        data = pd.get_dummies(data_frame[col], prefix=col.encode('ascii', 'replace'))
        all_columns = pd.concat([all_columns, data], axis=1)
    return all_columns

#find and remove variables with zero variance
def find_zero_var(df):
    """finds columns in the dataframe with zero variance -- ie those
        with the same value in every observation.
    """
    toKeep = []
    toDelete = []
    for col in df:
      if len(df[col].value_counts()) > 1:
         toKeep.append(col)
      else:
        toDelete.append(col)      
    return {'toKeep':toKeep, 'toDelete':toDelete}
    
#find and remove variables with perfect correlation
def find_perfect_corr(df):
    """finds columns that are eother positively or negatively perfectly 
        correlated (with correlations of +1 or -1), and creates a dict that 
        includes which columns to drop so that each remaining column is independent
    """
    corrMatrix = df.corr()
    corrMatrix.loc[:,:] = np.tril(corrMatrix.values, k = -1)
    already_in = set()
    result = []
    for col in corrMatrix:
        perfect_corr = corrMatrix[col][abs(np.round(corrMatrix[col],10)) == 1.00].index.tolist()
        if perfect_corr and col not in already_in:
            already_in.update(set(perfect_corr))
            perfect_corr.append(col)
            result.append(perfect_corr)
    toRemove = []
    for item in result:
        toRemove.append(item[1:(len(item)+1)])
        toRemove = sum(toRemove, [])
    return {'corrGroupings':result, 'toRemove':toRemove}

In [6]:
#GET DATA FROM SQL DB INTO PANDAS DATA FRAME-----------------------------------

#reconnect to SQLite DB
conn = sq.connect('/Users/harishkashyap/Documents/SQLite/lahman2013.sqlite')

#get position, dominant team and performance stats
query2 = """select h.*, 
  b.b_atbat, b.b_runs, b.b_hits, b.b_hruns, b.b_stbas, b.b_strik,
  p.p_wins, p.p_loss, p.p_shout, p.p_saves, p.p_eruns, p.p_stout, 
  f.f_puts, f.f_assis, f.f_dplay, f.f_pass, o.pos, t.teamid
from 
  (select playerid, max(case when inducted = 'Y' then 1 else 0 end) as inducted, max(yearid) as year
   from halloffame where category = 'Player' group by playerid) h
left outer join 
  (select playerid, sum(ab) as b_atbat, sum(r) as b_runs, sum(h) as b_hits, 
    sum(hr) as b_hruns, sum(sb) as b_stbas, sum(so) as b_strik
  from batting group by playerid) b on h.playerid = b.playerid
left outer join
  (select playerid, sum(w) as p_wins, sum(l) as p_loss, sum(sho) as p_shout,
    sum(sv) as p_saves, sum(er) as p_eruns, sum(so) as p_stout
  from pitching group by playerid) p on h.playerid = p.playerid
left outer join
  (select playerid, sum(po) as f_puts, sum(a) as f_assis, sum(dp) as f_dplay, sum(pb) as f_pass 
  from fielding group by playerid) f on h.playerid = f.playerid
left outer join
  (select * from dominant_position_per_player) o on h.playerid = o.playerid
left outer join
  (select * from dominant_team_per_player) t on h.playerid = t.playerid
;"""

df = pd.read_sql(query2, conn)

#close connection
conn.close()

#check data
df.head()
df.tail()
df.shape

#split data before and on/after year 2000 (training vs future predictions)
pre2000 = df[df.year < 2000.00]
post2000 = df[df.year >= 2000.00]

DatabaseError: Execution failed on sql: select h.*, 
  b.b_atbat, b.b_runs, b.b_hits, b.b_hruns, b.b_stbas, b.b_strik,
  p.p_wins, p.p_loss, p.p_shout, p.p_saves, p.p_eruns, p.p_stout, 
  f.f_puts, f.f_assis, f.f_dplay, f.f_pass, o.pos, t.teamid
from 
  (select playerid, max(case when inducted = 'Y' then 1 else 0 end) as inducted, max(yearid) as year
   from halloffame where category = 'Player' group by playerid) h
left outer join 
  (select playerid, sum(ab) as b_atbat, sum(r) as b_runs, sum(h) as b_hits, 
    sum(hr) as b_hruns, sum(sb) as b_stbas, sum(so) as b_strik
  from batting group by playerid) b on h.playerid = b.playerid
left outer join
  (select playerid, sum(w) as p_wins, sum(l) as p_loss, sum(sho) as p_shout,
    sum(sv) as p_saves, sum(er) as p_eruns, sum(so) as p_stout
  from pitching group by playerid) p on h.playerid = p.playerid
left outer join
  (select playerid, sum(po) as f_puts, sum(a) as f_assis, sum(dp) as f_dplay, sum(pb) as f_pass 
  from fielding group by pla